In [6]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model 
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, AveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from utility import preprocess_image
from keras.models import load_model 
import sys
import os
sys.path.insert(0, 'model')
import tensorflow as tf

In [7]:
#Get Training Config Parameters
run_dir = 'train_run'
nb_train_samples = 18190
nb_validation_samples = 2022
save_step = 5
epochs = 30000
batch_size = 8
learning_rate = 0.01
momentum = 0.99
fine_tuning_ture = 0
fine_tuning_model_path = "./train_run/XXX.hdf5"
initial_epoch=0

# GPU Usage Control
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [8]:
#prepare network
# base model selection
bm = 2
# dimensions of our images.
img_width, img_height = 100,100
num_classes = 452
#0 -- VGG16 (224x224)
#1 -- Mobilenet (224x224)
#2 -- CASIA (100*100)

#Strarting from random initialization
#if fine tuning from pre-trained model : weight = 'imagenet'
if(fine_tuning_ture == 0):
    #define input image size
    if K.image_data_format() == 'channels_first':
        input_shape = (1, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 1)
    
    #load predefined base network model
    if (bm==0):
        from keras.applications.vgg16 import VGG16 #VGG
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==1):
        from keras.applications.mobilenet import MobileNet #Mobilenet
        base_model = MobileNet(include_top=False, weights='imagenet', input_shape=input_shape)
    elif (bm==2):
        from casia import CASIA 
        base_model = CASIA(input_shape=input_shape)
    
    x = base_model.output
    
    if bm == 2:
        x = Flatten()(x)
    
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
else:
    model = load_model(fine_tuning_model_path)  

In [5]:
#para has problem
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 128)       0         
__________

In [9]:
# prepare ImageDataGenerator
train_data_dir = './dataset/lfw_mtcnnpy_160_gray_train_valid/train'
validation_data_dir = './dataset/lfw_mtcnnpy_160_gray_train_valid/valid'
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
#     horizontal_flip=True,
#     fill_mode='nearest',
#     rotation_range=20,
#     width_shift_range=0.9,
#     height_shift_range=0.9,
#     shear_range=0.2,
#     zoom_range=0.2,
    preprocessing_function=preprocess_image)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="grayscale",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = True)
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    color_mode="grayscale",
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle = True)

Found 18109 images belonging to 452 classes.
Found 2103 images belonging to 452 classes.


In [ ]:
#start to train
#compile model
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=learning_rate, momentum=momentum), metrics=['accuracy']    )

#setup check point
filepath=os.path.join(run_dir, "weights-improvement-{epoch:02d}-{val_loss:.2f}-{acc:.2f}-{val_acc:.2f}.hdf5")
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint(filepath,period=save_step)
callbacks_list = [checkpoint]

#start to train
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list,
    initial_epoch=initial_epoch)

Epoch 1/30000
2273/2273 [==============================] - 203s 89ms/step - loss: 15.7161 - acc: 0.0029 - val_loss: 16.0781 - val_acc: 0.0025
Epoch 2/30000
2273/2273 [==============================] - 247s 109ms/step - loss: 16.0782 - acc: 0.0025 - val_loss: 16.0781 - val_acc: 0.0025
Epoch 3/30000
2273/2273 [==============================] - 406s 179ms/step - loss: 16.0791 - acc: 0.0024 - val_loss: 16.0781 - val_acc: 0.0025
Epoch 4/30000
2273/2273 [==============================] - 350s 154ms/step - loss: 16.0773 - acc: 0.0025 - val_loss: 16.0781 - val_acc: 0.0025
Epoch 5/30000
2273/2273 [==============================] - 358s 158ms/step - loss: 16.0791 - acc: 0.0024 - val_loss: 16.0781 - val_acc: 0.0025
Epoch 6/30000
2273/2273 [==============================] - 329s 145ms/step - loss: 16.0755 - acc: 0.0026 - val_loss: 16.0781 - val_acc: 0.0025
Epoch 7/30000
2273/2273 [==============================] - 256s 112ms/step - loss: 16.0791 - acc: 0.0024 - val_loss: 16.0781 - val_acc: 0.0025
